In [40]:
%%sh
#!/usr/bin/env bash
docker system prune --force > /dev/null 2>&1

# The name of our algorithm
algorithm_name=wpi-test

cd testing

chmod -R +x coral/

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email) 2> /dev/null

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} . --no-cache # > /dev/null 2>&1
docker tag ${algorithm_name} ${fullname} > /dev/null 2>&1

docker push ${fullname} #> /dev/null 2>&1

Login Succeeded
Sending build context to Docker daemon  27.14kB
Step 1/11 : from jjanzic/docker-python3-opencv
 ---> 7b19919b069b
Step 2/11 : RUN mkdir work
 ---> Running in 73d9087a6833
Removing intermediate container 73d9087a6833
 ---> b5532e7a0fa5
Step 3/11 : ARG work_dir=/work
 ---> Running in ca767b80c6fa
Removing intermediate container ca767b80c6fa
 ---> 195e4b3ebf5f
Step 4/11 : RUN apt-get update -qq > /dev/null &&     apt-get install -y python python-tk python3 python3-pip -qq > /dev/null
 ---> Running in 07980d415fb4
debconf: delaying package configuration, since apt-utils is not installed
Removing intermediate container 07980d415fb4
 ---> 99994e64fbe9
Step 5/11 : RUN apt-get update -qq > /dev/null &&     apt-get install -y wget vim emacs nano -qq > /dev/null
 ---> Running in 8f2b87d12887
debconf: delaying package configuration, since apt-utils is not installed
Removing intermediate container 8f2b87d12887
 ---> fb2524077007
Step 6/11 : RUN apt-get install -y apt-transport-http